In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
pd.set_option('display.max_columns', 500)

In [2]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
#df9 = pd.read_csv('data/featureUsuarioRealizaAlMenos20EventosEn10Minutos.csv', low_memory=False, index_col='person')
#df10 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/featureUltimaConexion.csv', low_memory=False, index_col='person')

labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
#df_unidos = df_unidos.join(df9, how='inner')
#df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')

In [3]:
df_test.head()

,vio_productos,tiene_checkouts,tiene_brand_listing,tiene_generic_listing,tiene_visitas,llegaron_por_ad,llegaron_por_search,tiene_leads,tiene_busquedas,tiene_conversions,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_visitas,cant_leads,cant_brand_listings,cant_generic_listings,cant_entradas_ads,cant_entradas_buscador,total_sesiones,promedio_eventos_por_sesion,compro_Apple,compro_Asus,compro_LG,compro_Lenovo,compro_Motorola,compro_Quantum,compro_Samsung,compro_Sony,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony,dispositivo_Computer,dispositivo_Smartphone,dispositivo_Tablet,dispositivo_Unknown,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,compro_onsite,cant_eventos_4_a_10hs,cant_eventos_11_a_14hs,cant_eventos_15_a_20hs,cant_eventos_21_a_3hs,conte_10,EsDeSaoPablo,cantidadMaximaQueVeUnMismoProducto,timestamp,label
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ad93850f,True,True,True,True,True,True,True,False,False,False,0.0,1.0,20.0,0.0,5.0,0.0,15.0,7.0,10.0,7.0,5,13.000000,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0,1,0,0,1,0,0,0,0,0,0,False,0.0,0.0,0.0,65.0,True,True,17.0,20180522225054,0
1b9f7cf6,True,True,True,False,True,False,True,False,True,False,0.0,1.0,9.0,8.0,3.0,0.0,3.0,0.0,0.0,1.0,3,8.333333,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0,1,0,0,0,0,0,0,1,0,0,False,2.0,0.0,15.0,8.0,True,False,7.0,20180521101350,0
de8fe91b,True,True,True,True,True,False,True,False,True,False,0.0,1.0,27.0,13.0,2.0,0.0,3.0,3.0,0.0,4.0,2,26.500000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,1,0,0,0,1,0,0,0,0,0,0,False,0.0,0.0,0.0,53.0,True,True,11.0,20180521223728,0
45baf068,True,True,True,True,True,True,False,False,False,False,0.0,1.0,6.0,0.0,1.0,0.0,3.0,4.0,5.0,0.0,1,20.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0,1,0,0,1,0,0,0,0,0,0,False,0.0,0.0,0.0,20.0,True,False,5.0,20180518005448,0
99abca5a,True,True,True,True,True,True,True,True,True,True,2.0,18.0,630.0,5.0,56.0,1.0,30.0,62.0,12.0,1.0,49,16.714286,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,1,0,0,0,0,0,0,0,0,1,0,True,3.0,37.0,78.0,699.0,True,False,56.0,20180529211641,0


In [4]:
features = df_test.columns.tolist()
features.remove('label')

## Veamos si las labels están balanceadas

In [5]:
labels['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

# Entrenando el Random Forest

In [6]:
param_grid = { 
    'n_estimators': [x for x in range(300, 500, 50)],
    'max_depth' : [6,7,8,9],
    'min_samples_split': [x for x in range(200, 400, 50)]
}

In [7]:
#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20, random_state=80)

In [35]:
rfc = RandomForestClassifier(class_weight='balanced', random_state=80)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=4)
CV_rfc.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=80,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [300, 350, 400, 450], 'max_depth': [6, 7, 8, 9], 'min_samples_split': [200, 250, 300, 350]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [36]:
CV_rfc.best_params_

{'max_depth': 7, 'min_samples_split': 300, 'n_estimators': 350}

In [82]:
# Submit de 83 puntos
#  class_weight='balanced' hace que se le de la misma importancia a las clases a predecir, sin importar (creo)
#  su distribución.
#  Los valores de los parámetros los seleccioné probando, aún no sé bien cómo influencian los resultados.
#rf = RandomForestClassifier(n_estimators=250, n_jobs=2, min_samples_split=200,\
#                             random_state=80, class_weight='balanced')

In [8]:
rf = RandomForestClassifier(n_estimators=3000, n_jobs=-1, min_samples_split=300,\
                            max_depth=7, random_state=80, class_weight='balanced')
rf.fit(X_train,Y_train)
Y_pred = rf.predict_proba(X_test)
Y_pred_proba = [p[1] for p in Y_pred]
print(roc_auc_score(Y_test, Y_pred_proba))

0.8417146473012945


In [9]:
feature_importances = pd.DataFrame(rf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)
#  Acá quedan ordenadas las features por importancia para el modelo, aparentemente la marca más vista y la marca
#  que más compró cada usuario no es tan importante.
feature_importances.head(40)

,importance
cant_checkouts,0.230495
tiene_checkouts,0.226656
timestamp,0.087592
total_sesiones,0.039777
cant_viewed_product,0.033929
cant_visitas,0.033713
cant_brand_listings,0.025601
cant_entradas_buscador,0.025136
cant_generic_listings,0.023897
cant_conversions,0.022607


# Preparamos el submit

In [10]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [11]:
df_events.head()

,vio_productos,tiene_checkouts,tiene_brand_listing,tiene_generic_listing,tiene_visitas,llegaron_por_ad,llegaron_por_search,tiene_leads,tiene_busquedas,tiene_conversions,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_visitas,cant_leads,cant_brand_listings,cant_generic_listings,cant_entradas_ads,cant_entradas_buscador,total_sesiones,promedio_eventos_por_sesion,compro_Apple,compro_Asus,compro_LG,compro_Lenovo,compro_Motorola,compro_Quantum,compro_Samsung,compro_Sony,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony,dispositivo_Computer,dispositivo_Smartphone,dispositivo_Tablet,dispositivo_Unknown,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,compro_onsite,cant_eventos_4_a_10hs,cant_eventos_11_a_14hs,cant_eventos_15_a_20hs,cant_eventos_21_a_3hs,conte_10,EsDeSaoPablo,cantidadMaximaQueVeUnMismoProducto,timestamp
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,True,True,False,True,True,False,True,False,True,False,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1,9.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0,1,0,0,1,0,0,0,0,0,0,False,0.0,0.0,0.0,9.0,True,False,4.0,20180518003030
0297fc1e,True,True,True,True,True,True,False,True,True,False,0.0,7.0,404.0,6.0,95.0,1.0,4.0,21.0,29.0,0.0,89,6.370787,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0,1,0,0,0,1,0,0,0,0,0,False,37.0,120.0,199.0,187.0,True,False,213.0,20180528115012
2d681dd8,True,True,True,True,True,True,True,False,True,False,0.0,1.0,13.0,1.0,2.0,0.0,5.0,1.0,1.0,2.0,2,13.000000,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,1,0,0,0,0,0,0,1,0,0,0,False,0.0,0.0,16.0,10.0,False,False,6.0,20180527194249
cccea85e,True,True,True,True,True,True,True,False,True,False,0.0,1.0,739.0,1.0,22.0,0.0,7.0,20.0,15.0,26.0,17,49.176471,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,1,0,0,0,0,1,0,0,0,0,0,False,0.0,77.0,241.0,518.0,True,False,252.0,20180531143830
4c8a8b93,True,True,True,True,True,True,True,False,True,False,0.0,2.0,177.0,9.0,20.0,0.0,8.0,14.0,14.0,13.0,14,18.357143,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0,1,0,0,1,0,0,0,0,0,0,False,0.0,66.0,17.0,174.0,True,False,83.0,20180522003318


In [12]:
kaggle_pred = rf.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [13]:
df_submit.to_csv('submit.csv')